In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO('yolov8n.pt')
cap = cv2.VideoCapture('wiiplay.mp4')

start_frame = 41000
end_frame = 44000

cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

max_people_count = 0
frame_with_max_people = None

while True:
    ret, frame = cap.read()
    if ret == False:
        break
        
    results = model.predict(frame, conf=0.20)

    people_count = 0 # 初始化人數
    for result in results:
        for box in result.boxes:
            label = results[0].names[int(box.cls)]
            if label == 'person':
                people_count += 1
                left, top, right, bottom = np.array(box.xyxy, dtype=np.uint16).squeeze() #convert from tensor to list
                confidence = float(box.conf.cpu())
                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                cv2.putText(frame, label+' '+'{:.2f}'.format(confidence),(left+5, bottom-10),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)

    # 計算最大人數的frame
    if people_count > max_people_count:
        max_people_count = people_count
        frame_with_max_people = frame.copy()
        
    # 在左上角顯示人數總數
    cv2.putText(frame, f'Persons: {people_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Output', frame)
    
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == end_frame:
        break
    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()

# 顯示最多人數的frame
if frame_with_max_people is not None:
    cv2.putText(frame_with_max_people, f'Max People:{max_people_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Frame with Most People', frame_with_max_people)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

D:\anaconda3\envs\Computer_Vision\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
